In [2]:
import pandas as pd
import os

In [1]:
aziende_mapping = {
    # Health Districts (ULSS)
    "501": "AZIENDA ULSS N. 1 DOLOMITI",
    "502": "AZIENDA ULSS N. 2 MARCA TREVIGIANA",
    "503": "AZIENDA ULSS N. 3 SERENISSIMA",
    "504": "AZIENDA ULSS N. 4 VENETO ORIENTALE",
    "505": "AZIENDA ULSS N. 5 POLESANA",
    "506": "AZIENDA ULSS N. 6 EUGANEA",
    "507": "AZIENDA ULSS N. 7 PEDEMONTANA",
    "508": "AZIENDA ULSS N. 8 BERICA",
    "509": "AZIENDA ULSS N. 9 SCALIGERA",
    
    # Cities/Locations
    "101": "BELLUNO",
    "102": "FELTRE",
    "103": "BASSANO DEL GRAPPA",
    "104": "THIENE",
    "105": "ARZIGNANO",
    "106": "VICENZA",
    "107": "PIEVE DI SOLIGO",
    "108": "ASOLO",
    "109": "TREVISO",
    "110": "S. DONA' DI PIAVE",
    "112": "VENEZIANA",
    "113": "MIRANO",
    "114": "CHIOGGIA",
    "115": "CITTADELLA",
    "116": "PADOVA",
    "117": "ESTE",
    "118": "ROVIGO",
    "119": "ADRIA",
    "120": "VERONA",
    "121": "LEGNAGO",
    "122": "BUSSOLENGO",
    
    # University Hospitals
    "901": "AZIENDA OSPEDALE - UNIVERSITÀ PADOVA",
    "912": "A.O. INTEGRATA CON UNIVERSITÀ - VERONA"
}

anni_mapping = {
    1: 2015,
    2: 2016,
    3: 2017,
    4: 2018,
    5: 2019,
    6: 2020,
    7: 2021,
    8: 2022,
    9: 2023
}


In [4]:
# Get all CSV files in the raw directory
raw_dir = '../output/raw/'
clean_dir = '../output/clean/'

In [20]:
# Process each CSV file
for filename in os.listdir(raw_dir):
    if filename.endswith('.csv'):
        print(f"\nProcessing {filename}")
        try:
            # Read the CSV
            df = pd.read_csv(os.path.join(raw_dir, filename))
            print(f"Original columns: {df.columns.tolist()}")
            
            # Apply the transformations
            df['anno'] = df['anno'].map(anni_mapping)
            df['nome'] = df['nome_azienda'].astype(str).map(aziende_mapping)
            df['valori'] = df['valori'].astype(int)

            # Check if 'categoria' column exists
            if 'categoria' in df.columns:
                print(f"Applying pivot to {filename} - shape before pivot: {df.shape}")
                # Apply pivot
                df = df.pivot_table(
                    index=['anno', 'nome'], 
                    columns='categoria', 
                    values='valori',
                    aggfunc='first'
                ).reset_index()
                print(f"Shape after pivot: {df.shape}")

            # Clean up
            df = df.fillna(0)
            df.columns = df.columns.str.strip()
            df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace("'", '').str.replace('"', '').str.strip()
            df['nome'] = df['nome'].str.title().str.strip()
            
            print(f"Final columns: {df.columns.tolist()}")
            
            # Save processed file
            clean_filename = filename.replace('.csv', '_clean.csv')
            output_filename = os.path.join(clean_dir, clean_filename)

            df.to_csv(output_filename, index=False, encoding='utf-8')
            print(f"Saved processed file to {clean_filename}")
            print(f"Final shape: {df.shape}")
            
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            print(f"Error occurred at file: {filename}")
            print(f"Current DataFrame state:\n{df.head()}")

print("Processing complete!")


Processing ivg_per_'mifepristone'_e_'mifepristone_più_prostaglandina'_e_altro_intervento_più_diffuso_e_terapia_antalgica.csv
Original columns: ['nome_azienda', 'nome', 'categoria', 'valori', 'anno']
Applying pivot to ivg_per_'mifepristone'_e_'mifepristone_più_prostaglandina'_e_altro_intervento_più_diffuso_e_terapia_antalgica.csv - shape before pivot: (568, 5)
Shape after pivot: (118, 8)
Final columns: ['anno', 'nome', 'altra', 'analgesia_senza_anestesia', 'anestesia_generale', 'anestesia_locale', 'nessuna', 'sedazione_profonda']
Saved processed file to ivg_per_'mifepristone'_e_'mifepristone_più_prostaglandina'_e_altro_intervento_più_diffuso_e_terapia_antalgica_clean.csv
Final shape: (118, 8)

Processing ivg_per_tempo_di_attesa_per_la_certificazione.csv
Original columns: ['nome_azienda', 'categoria', 'valori', 'anno']
Applying pivot to ivg_per_tempo_di_attesa_per_la_certificazione.csv - shape before pivot: (492, 5)
Shape after pivot: (123, 6)
Final columns: ['anno', 'nome', '29_giorni_